In [1]:
%matplotlib inline
import numpy as np
import jax.numpy as jnp
import sys
sys.path.append('../bayesianquilts')
import json
import matplotlib.pyplot as plt
import uuid
import jax
from factor_analyzer import FactorAnalyzer

jax.config.update("jax_enable_x64", True)


In [2]:


from autoencirt.irt import GRModel, FactorizedGRModel
from autoencirt.data.rwa import item_text, get_data, to_reverse

dim = 2

In [3]:
pd_data = get_data(reorient=True, pandas=True)
responses = pd_data[0].iloc[:, :22]
fa = FactorAnalyzer(n_factors=dim)
fa.fit(responses)
loadings = fa.loadings_
loadings.shape

(22, 2)

In [4]:
dataset, num_people = get_data(reorient=True)


In [5]:
batch_size=241

In [11]:
def data_factory_factory(batch_size, repeat=False, shuffle=False):
    def dfactory(batch_size=batch_size):
        ds = dataset.shuffle(seed=10) if shuffle else dataset
        ds = ds.repeat() if repeat else ds
        return dataset.batch(batch_size=batch_size).repeat()
    return dfactory

In [12]:
discrimination_init = 0.5*jnp.abs(loadings).T.astype(jnp.float64)[
        jnp.newaxis, :, :, jnp.newaxis
    ]



In [13]:
item_names = [f"Q{j}" for j in range(1, 23)]
scale_indices = [
    [1, 3, 5, 7, 8, 10, 12, 14, 15, 17, 19, 20],
    [0, 2, 4, 6, 9, 11, 13, 16, 18, 21],
]
fgrm = FactorizedGRModel(
    data=dataset,
    item_keys=item_names,
    num_people=num_people,
    dim=dim,
    eta_scale=1e-3,
    kappa_scale=1e-3,
    weight_exponent=1,
    response_cardinality=9,
    scale_indices=scale_indices,
    discrimination_guess=discrimination_init,
    dtype=jnp.float64
)


In [14]:
p = fgrm.sample(20)

In [ ]:
losses, final_params = fgrm.fit(
    data_factory_factory(batch_size=batch_size, shuffle=True, repeat=True),
    dataset_size=num_people,
    accumulation_steps=1,
    steps_per_epoch=num_people//batch_size,
    batch_size=batch_size,
    num_epochs=1000,
    check_convergence_every=10,
    patience=5,
    learning_rate=1e-4,
    lr_decay_factor=0.99,
    checkpoint_dir=None #f"/tmp/autoenc{uuid.uuid4()}"
)

--- Starting Training ---
Patience for early stopping: 5 epochs
LR decay factor on plateau: 0.99
Convergence will be checked every: 10 epoch(s)
Checkpoints will be saved to: None
-------------------------


Epoch 1/1000 (LR: 0.000100):   0%|          | 0/41 [00:00<?, ?batch/s]